O codigo , busca no site do Inep os dados dos respectivos anos atualizado , faz o donwload das pastas .zip e extrai tudo em uma pasta denominada de enade, pos isso as tabelas com todos os dados são atribuidas a uma variavel denominada inicalmente com "enade" sequido do ano, por exemplo enade2017 para os dados separados de 2017.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import requests
from io import BytesIO
import os
import plotly.graph_objs as go 
import plotly.offline as pyo

os.makedirs("./enade", exist_ok=True)

url1="https://download.inep.gov.br/microdados/Enade_Microdados/microdados_enade_2019.zip" 
url2="https://download.inep.gov.br/microdados/Enade_Microdados/microdados_enade_2018.zip"
url3="https://download.inep.gov.br/microdados/Enade_Microdados/microdados_Enade_2017_portal_2018.10.09.zip"

filebytes= BytesIO(requests.get(url1).content)
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./enade")
filebytes= BytesIO(requests.get(url2).content)
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./enade")
filebytes= BytesIO(requests.get(url3).content)
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./enade")

enade2017 = pd.read_csv("/content/enade/3.DADOS/MICRODADOS_ENADE_2017.txt",sep = ";", decimal = ",")
enade2018 = pd.read_csv("/content/enade/2018/3.DADOS/microdados_enade_2018.txt",sep = ";", decimal = ",")
enade2019 = pd.read_csv("/content/enade/3.DADOS/microdados_enade_2019.txt",sep = ";", decimal = ",")

Após a coleta dos dados eles são tratados primeiramente concatenando todas as tabelas em uma unica tabela base para tratamento , em sequida é feita uma filtragem de colunas selecionadas e criada uma nova coluna utilizando o index de linha representando o codigo de incricão do estudante 

In [ ]:
enadeBase = enade2017
enadeBase = enadeBase.append(enade2018)
enadeBase = enadeBase.append(enade2019)

In [ ]:
enadeBase=enadeBase.iloc[:,[0,1,4,8,9,10,11,33,44,60,61,70,76,77,79]]
enadeBase.reset_index(level=0, inplace=True)
enadeBase = enadeBase.rename(columns={'index': 'Codigo inscrição do estudante'})

In [ ]:
enadeBase['CO_REGIAO_CURSO'] = enadeBase['CO_REGIAO_CURSO'].map({1:'Norte (NO)',2:'Nordeste (NE)',3:'Sudeste (SE)',4:'Sul (SUL)',5:'Centro-Oeste (CO)'},na_action=None)
enadeBase['QE_I02'] = enadeBase['QE_I02'].map({"A":'Branca',"B":'Preta',"C":'Amarela',"D":'Parda',"E":'Indígena',"F":"Não Declarada"},na_action=None)
enadeBase['CO_UF_CURSO'] = enadeBase['CO_UF_CURSO'].map({11:'Rondônia (RO)',12:'Acre (AC)',13:'Amazonas (AM)',14:'Roraima (RR)',15:'Pará (PA)',16:'Amapa (AP)',17:'Tocantins (TO)',21:'Maranhão (MA)',22:'Piauí (PI)',23:'Ceará (CE)',24:'Rio Grande do Norte (RN)',25:'Paraíba (PB)',26:'Pernambuco (PE)',27:'Alagoas (AL)',28:'Sergipe (SE)',29:' Bahia (BA)',31:'Minas gerais (MG)',32:'Espírito Santo (ES)',33:' Rio de Janeiro (RJ)',35:'São Paulo (SP)',41:' Paraná (PR)',42:' Santa Catarina (SC)',43:' Rio Grande do Sul (RS)',50:'Mato Grosso do Sul (MS)',51:'Mato Grosso (MT)',52:'Goiás (GO)',53:'Distrito federal (DF)'},na_action=None)
enadeBase['TP_PRES'] = enadeBase['TP_PRES'].map({222:'Ausente',334:'Eliminado',444:'Ausente',555:'Presente',556:'Eliminado'},na_action=None)
enadeBase['CO_RS_I1'] = enadeBase['CO_RS_I1'].map({'A':'Muito Fácil','B':'Fácil','C':'Médio','D':'Difícil','E':'Muito Difícil','.':'Sem resposta/Resposta anulada','*':'Sem resposta/Resposta anulada'},na_action=None)
enadeBase['CO_RS_I2'] = enadeBase['CO_RS_I2'].map({'A':'Muito Fácil','B':'Fácil','C':'Médio','D':'Difícil','E':'Muito Difícil','.':'Sem resposta/Resposta anulada','*':'Sem resposta/Resposta anulada'},na_action=None)
enadeBase['QE_I11'] = enadeBase['QE_I11'].map({'A':'Nenhuma, Curso Gratuito','B':'Nenhuma, Curso Pago','C':'ProUni integra','D':'ProUni parcial','E':'FIES','F':'ProUni Parcial e FIES','G':'Bolsa do estado, distrito ou municipio','H':'Bolsa da própria instituição','I':'Bolsa de entidades não governamentais','J':'Financiamento da própria instituição','K':'Financiamento bancário',np.nan: "Não Respondido"},na_action=None)

In [ ]:
DadosEstudante=enadeBase.iloc[:,[6,7,12]]
Bolsa=enadeBase.iloc[:,[14]]
Presença=enadeBase.iloc[:,[8]]
Localidade=enadeBase.iloc[:,[4,5]]
difEnade=enadeBase.iloc[:,[10,11]]
Instituição=enadeBase.iloc[:,[2]]
Ano=enadeBase.iloc[:,[1]]
Curso=enadeBase.iloc[:,[3]]
Notas=enadeBase.iloc[:,[9]]
Renda=enadeBase.iloc[:,[13]]


In [ ]:
import sqlite3
from sqlite3 import Error
conn = sqlite3.connect('BancoBase.db')
enadeBase.to_sql('TabelaFato',conn,if_exists='replace',index=False)
DadosEstudante.to_sql('CD_Estudante',conn,if_exists='replace',index=False)
Bolsa.to_sql('CD_Bolsa',conn,if_exists='replace',index=False)
Presença.to_sql('CD_Presença',conn,if_exists='replace',index=False)
Localidade.to_sql('CD_Localidade',conn,if_exists='replace',index=False)
difEnade.to_sql('CD_difEnade',conn,if_exists='replace',index=False)
Instituição.to_sql('CD_Instituição',conn,if_exists='replace',index=False)
Ano.to_sql('CD_Ano',conn,if_exists='replace',index=False)
Curso.to_sql('CD_Curso',conn,if_exists='replace',index=False)
Notas.to_sql('CD_Notas',conn,if_exists='replace',index=False)
Renda.to_sql('CD_Renda',conn,if_exists='replace',index=False)

In [ ]:
BolsasAno = enadeBase.iloc[:,[1,14]]


In [ ]:
ano=BolsasAno.groupby(BolsasAno.NU_ANO)
Ano2017 = ano.get_group(2017)
Ano2018=ano.get_group(2018)
Ano2019=ano.get_group(2019)
Ano2017=Ano2017.groupby('QE_I11').count()
Ano2018=Ano2018.groupby('QE_I11').count()
Ano2019=Ano2019.groupby('QE_I11').count()
Ano2017 = Ano2017.rename(columns={'NU_ANO': '2017'})
Ano2018 = Ano2018.rename(columns={'NU_ANO': '2018'})
Ano2019 = Ano2019.rename(columns={'NU_ANO': '2019'})
Ano2017 = Ano2017.transpose()
Ano2018 = Ano2018.transpose()
Ano2019 = Ano2019.transpose()

In [ ]:
Anos = Ano2017
Anos = Anos.append(Ano2018)
Anos = Anos.append(Ano2019)
Anos.reset_index(level=0, inplace=True)
Anos = Anos.rename(columns={'index': 'Anos'})

In [ ]:
Grafico = go.Figure(
   data=[
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Nenhuma, Curso Gratuito'],
           name='Nenhuma, Curso Gratuito',
           marker=dict(color='red')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Nenhuma, Curso Pago'],
           name='Nenhuma, Curso Pago',
           marker=dict(color='green')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['ProUni integra'],
           name='ProUni integra',
           marker=dict(color='blue')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['FIES'],
           name='FIES',
           marker=dict(color='orange')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['ProUni Parcial e FIES'],
           name='ProUni Parcial e FIES',
           marker=dict(color='yellow')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Bolsa do estado, distrito ou municipio'],
           name='Bolsa do estado, distrito ou municipio',
           marker=dict(color='purple')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Bolsa da própria instituição'],
           name='Bolsa da própria instituição',
           marker=dict(color='gray')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Bolsa de entidades não governamentais'],
           name='Bolsa de entidades não governamentais',
           marker=dict(color='pink')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Financiamento da própria instituição'],
           name='Financiamento da própria instituição',
           marker=dict(color='coral')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Financiamento bancário'],
           name='Financiamento bancário',
           marker=dict(color='brown')
       ),
       go.Bar(
           x=Anos['Anos'],
           y=Anos['Não Respondido'],
           name='Não Respondeu',
           marker=dict(color='black')
       ),
   ],
   layout=go.Layout(
       
   )
)
pyo.plot(Grafico)

In [ ]:
Grafico